**Welcome to the KoboldAI Colab Service!**<br/>
*Note: This colab is intended to be used with the KoboldAI Client, [which can be downloaded from GitHub here](https://github.com/KoboldAI/KoboldAI-Client).*

**Things you will need:**<br/>
- A Google Drive account
- One of the following models:
   - The GPT-Neo-2.7B-Horni model from [mega](https://mega.nz/file/6BNykLJb#B6gxK3TnCKBpeOF1DJMXwaLc_gcTcqMS0Lhzr1SeJmc), [gdrive](https://drive.google.com/file/d/1-Jj_hlyNCQxuSnK7FFBXREGnRSMI5MoF/view?usp=sharing), or [torrent](https://tinyurl.com/pytorch-gptneo-horni)
   - The GPT-Neo-2.7B-Horni-Ln model from [mega](https://mega.nz/file/rQcWCTZR#tCx3Ztf_PMe6OtfgI95KweFT5fFTcMm7Nx9Jly_0wpg), [gdrive](https://drive.google.com/file/d/1M1JY459RBIgLghtWDRDXlD4Z5DAjjMwg/view?usp=sharing), or [torrent](https://tinyurl.com/pytorch-gptneo-horni)

**Preparation:**<br/>
Download the model and place the .tar file in the root folder of your Google Drive. If you want to skip the unpacking stage of setup, extract the .tar to a folder in the root of your GDrive ***with the same name as the tar file.***<br/>
(e.g. **gpt-neo-2.7B-horni.tar** > extract to folder named **gpt-neo-2.7B-horni**)

In [ ]:
#@title <b>Step 1 - Install Dependencies</b>
#@markdown Press the Play button and wait for the script to finish.
from IPython.display import clear_output
from termcolor import colored
 
!pip install flask-ngrok
!pip install git+https://github.com/finetuneanon/transformers@gpt-neo-dungeon-localattention2
!pip install termcolor
!pip install flask_cloudflared
clear_output()
print(colored("DONE!", "green"))

In [ ]:
#@title <b>Step 2 - Adjust Your Settings</b>
#@markdown 1. Choose which model you have downloaded:
model_name = "2.7B-horni" #@param ["2.7B-horni", "2.7B-horni-ln", "EleutherAI/gpt-neo-2.7B"]
#@markdown 2. Is it still archived (.tar) or did you extract it? (Ignore for stock Neo)
is_archived = "It's archived." #@param ["It's archived.", "I extracted it."]
#@markdown 3. Connect via Ngrok or Cloudflare?
connect_method = "Ngrok" #@param ["Ngrok", "Cloudflare"]
#@markdown 4. Run in half-precision mode? (Use half if you are getting CUDA out-of-memory errors.)
precision = "Half" #@param ["Half", "Full"]
#@markdown 5. Press Play button to lock in settings <b>(Do not skip!)</b>

In [ ]:
#@title <b>Step 3 - Run Service</b>
#@markdown Press the Play button. You will be asked to link your 
#@markdown Google Drive account to this colab. Click the link that 
#@markdown pops up, give Drive permission at the prompt, then copy 
#@markdown the access code that's presented back into this notebook.
#@markdown <br/><br/>
#@markdown This notebook will then extract the tar file (if necessary) 
#@markdown and initialize the AI model. <b>This will take several minutes.</b>
#@markdown When the model is ready, Flask will start and give you a 
#@markdown Cloudflare or Ngrok address which looks like this:<br/>
#@markdown <i>https://\<unique id\>.trycloudflare.com/</i><br/>
#@markdown <i>http://\<unique id\>.ngrok.io/</i><br/>
#@markdown You will need to right-click this and copy the address.
#@markdown Start the KoboldAI Client on your computer and choose 
#@markdown Google Colab as the model. You will be asked to paste 
#@markdown the copied address into the terminal.
from flask import Flask, redirect, url_for, request
import json
import torch
import requests
import subprocess
from transformers import GPTNeoForCausalLM, AutoTokenizer, pipeline
import tarfile
from google.colab import drive
import os
import re
import time
from threading import Timer

if connect_method == "Cloudflare":
   from flask_cloudflared import run_with_cloudflared
elif connect_method == "Ngrok":
   from flask_ngrok import run_with_ngrok

# Thanks to finetune for some of this startup code, I'm really not
# familiar with the Colab environment
model         = None
tokenizer     = None
custom_models = ["2.7B-horni", "2.7B-horni-ln"]
is_archived   = True if is_archived == "It's archived." else False

# Get access to the unpacked model folder
if model_name in custom_models:
   # Get connected to Google Drive
   print(colored("Requesting Google Drive access...", "magenta"))
   drive.mount('/content/drive/')
   if is_archived:
      # Archived. Set path to tar file and unpack it
      model_gdrive = "/content/drive/MyDrive/gpt-neo-{0}.tar".format(model_name)
      if not os.path.isdir("gpt-neo-"+model_name):
         print(colored("Unpacking tar file, please wait...", "magenta"))
         tar = tarfile.open(model_gdrive, "r")
         tar.extractall()
         tar.close()
         print(colored("DONE!", "green"))
   else:
      # Unpacked model already available, just set the path to it
      model_gdrive = "/content/drive/MyDrive/gpt-neo-{0}".format(model_name)

# Initialize the model
print(colored("Initializing model, please wait...", "magenta"))

if model_name in custom_models:
   if is_archived:
      # If the model was archived, it now lives in the Colab's /content directory
      checkpoint = torch.load("gpt-neo-" + model_name + "/pytorch_model.bin", map_location="cuda:0")
      model = GPTNeoForCausalLM.from_pretrained("gpt-neo-" + model_name, state_dict=checkpoint).half().to("cuda").eval()
   else:
      # The unpacked folder lives on the user's GDrive
      checkpoint = torch.load(model_gdrive + "/pytorch_model.bin", map_location="cuda:0")
      model = GPTNeoForCausalLM.from_pretrained(model_gdrive, state_dict=checkpoint).half().to("cuda").eval()
else:
   from transformers.file_utils import cached_path, WEIGHTS_NAME, hf_bucket_url
   archive_file = hf_bucket_url(model_name, filename=WEIGHTS_NAME)
   resolved_archive_file = cached_path(archive_file)
   checkpoint = torch.load(resolved_archive_file, map_location="cuda:0")
   for k in checkpoint.keys():
      checkpoint[k] = checkpoint[k].half()
   model = GPTNeoForCausalLM.from_pretrained(model_name, state_dict=checkpoint).half().to("cuda").eval()

for k in list(checkpoint.keys()):
   del checkpoint[k]
del checkpoint

# Initialize the tokenizer and set up the bad_words_ids to exclude Author's Note tags
tokenizer = AutoTokenizer.from_pretrained("gpt2")
vocab         = tokenizer.get_vocab()
vocab_keys    = vocab.keys()
find_keys     = lambda char : [key for key in vocab_keys if key.find(char) != -1]
bad_words     = []
bad_words_ids = []

bad_words.extend(find_keys("["))
bad_words.extend(find_keys(" ["))
for key in bad_words:
  bad_id = vocab[key]
  bad_words_ids.append([bad_id])

# Enable 32-bit mode if the GPU can handle it
if precision == "Full":
  if torch.cuda.get_device_properties(0).total_memory > 15000 * 1024 * 1024:
    print(colored("Big GPU detected, using fp32", "magenta"))
    model = model.float()

print(colored("DONE!", "green"))

app = Flask(__name__)

if connect_method == "Cloudflare":
   run_with_cloudflared(app)
elif connect_method == "Ngrok":
   run_with_ngrok(app)

@app.route("/")
def home():
    return "<h1>KoboldAI Colab Service Running!</h1>"

@app.route('/request',methods = ['POST'])
def koboldrequest():
   if request.method == 'POST':
      try:
        clear_output()
        js      = request.json
        txt     = js["text"]
        min     = js["min"]
        max     = js["max"]
        rep_pen = js["rep_pen"]
        temp    = js["temperature"]
        top_p   = js["top_p"]

        # Compatability with un-updated clients
        if("numseqs" in js):
          numseqs = js["numseqs"]
        else:
          numseqs = 1

        if("retfultxt" in js):
          retfultxt = js["retfultxt"]
        else:
          retfultxt = True

        print(colored("Received Data: {0}".format(txt), "yellow"))

        torch.cuda.empty_cache()
        print(colored("Generating text, please wait...", "green"))

        tokens = tokenizer(txt, return_tensors="pt").input_ids.to("cpu")
        ids = tokens.cuda()

        gen_tokens = model.generate(
              ids.long().cuda(),
              do_sample=True,
              min_length=min,
              max_length=max,
              temperature=temp,
              top_p = top_p,
              repetition_penalty = rep_pen,
              use_cache=True,
              bad_words_ids=bad_words_ids,
              num_return_sequences=numseqs
          ).long()

        genout = []
        for tkns in gen_tokens:
          if(not retfultxt):
            # Strip context tokens out of returned sequences
            dif = (len(tkns) - len(tokens[0])) * -1
            tkns = tkns[dif:]
          genout.append(tokenizer.decode(tkns))
        torch.cuda.empty_cache()

        if(len(genout) > 0 and genout[0] != ""):
          if(retfultxt):
            # Outdated client, send old JSON format
            print(colored("Generated Text: {0}".format(genout[0]), "cyan"))
            response = app.response_class(
              response=json.dumps({"data": {"text": genout[0]}}),
              status=200,
              mimetype='application/json'
            )
          else:
            # New client format with numseq support
            i = 0
            for seq in genout:
              print(colored("[Result {0}]\n{1}".format(i, seq), "cyan"))
              i += 1
            response = app.response_class(
              response=json.dumps({"data": {"seqs": genout}}),
              status=200,
              mimetype='application/json'
            )

          return response
        else:
          print(colored("[ERROR] Something went wrong during generation!", "red"))
          response = app.response_class(
            response=json.dumps({"error": {"extensions": {"code": "Something went wrong during generation!"}}}),
            status=400,
            mimetype='application/json'
          )
        
        js         = {}
        tokens     = []
        ids        = []
        gen_tokens = []
        genout     = ""
        response   = {}

      except Exception as e:
        print(colored("[ERROR] Something went wrong during generation!", "red"))
        print(colored("{0}".format(e), "red"))
        response = app.response_class(
          response=json.dumps({"error": {"extensions": {"code": "Something went wrong during generation!"}}}),
          status=400,
          mimetype='application/json'
        )

print(colored("Starup complete! Running web service.", "green"))
app.run()